<a href="https://colab.research.google.com/github/nalivaikaanastasiya-dev/Agentic-RAG-Streamlit-App/blob/main/agentic_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from agent_rag_pipeline import AgenticRAG
from langchain.document_loaders import TextLoader

# Загружаем примеры документов
loader = TextLoader("example_docs.txt")  # простой текстовый файл с KB
docs = loader.load()

# Инициализируем агент
agent = AgenticRAG(docs)

# Пример запросов
queries = [
    "What is the agentic loop?",
    "When should the agent use tools?",
    "Cite the sources."
]

for q in queries:
    print(f"Q: {q}")
    print(agent.get_answer(q))
    print("---")
